Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer  # Or use a lemmatizer if preferred
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
import torch
from torch.utils.data import Dataset, DataLoader
import openpyxl
!pip install datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

Load Data

In [2]:
from datasets import DatasetDict
from datasets import load_dataset

dataset = load_dataset("barolr/text_am-sum")

ds_train_devtest = dataset['train'].train_test_split(test_size=0.01, seed=42)

df = pd.DataFrame(ds_train_devtest['train'])
df

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,title,content,link,id
0,ነፃ ያልወጣች ሠንደቅ (መስፍን ማሞ ተሰማ),\n233\n\n\nአዎ፤ አውሮፓዊው ቅኝ ገዢ ጣሊያን አድዋ ላይ ድል ሆኗል...,https://www.ethiopianregistrar.com/amharic/pag...,181
1,title,content,link,id
2,ግዕዝ ኦሮምኛ ጨምሮ ለሁሉም የኢትዮጵያ ቋንቋዎች ብቃት እንዳለው ዶ/ር አ...,\n331\n\n\n“አገርኛ ቋንቋን ከኮምፒዩተር ያዋሃዱ ሳይንቲስት!” በሚ...,https://www.ethiopianregistrar.com/amharic/pag...,313
3,ከ OPDO ማዕከላዊ ኮሚቴ አፈትልከው የወጡት መረጃዎች እንደሚጠቁሙት አቶ...,\n2.1k\n\n\nጥር 26 ቀን 2008 ዓ.ም\nእንደ OPDO የውስጥ ም...,https://www.ethiopianregistrar.com/amharic/pag...,201
4,ባሕር ዳር ላይ በተደረገው የአፍሪካ ዋንጫ ማጣሪያ ጨዋታ ኢትዮጵያ ኮትዲቯ...,\n361\n\n\n\n\nጨዋታውን አስመልክቶ አሰልጣኙ፣ ተጨዋቾች እና ደጋ...,https://www.ethiopianregistrar.com/amharic/pag...,33
...,...,...,...,...
24093,“ከዚህ በፊት የኢትዮጵያ እስር ቤቶች ይታወቁ የነበረው በአካል ላይ ጥቃት...,\n753\n\n\n• ኢትዮጵያ ውስጥ ከአንድ ዓመት በፊት አዲስ የፖለቲከ ...,https://www.ethiopianregistrar.com/amharic/pag...,30
24094,ለባልደራስ፣ለኢዜማ፣ ለኦሮሞ ፓርቲ፣ ወይንስ ለፒፒ(ደኢህዴን)? የጉራጌ ህ...,\n361\n\n\nበ ሸምሱ በረዳ\n\nእንደሚታወቀው ቀጣዩ ሃገር አቀፍ ም...,https://www.ethiopianregistrar.com/amharic/pag...,65
24095,ሁለቱ ዋርዲያዎች፤ ጆቤና ጻድቃን ገብረተንሳይ በባህር በር አጀንዳ – መ...,\n501\n\n\nከተወሰ ግዜ ወዲህ አበበ ተክለ ሃይማኖት እና ጻድቃን ስ...,https://www.ethiopianregistrar.com/amharic/pag...,76
24096,ታላቁ ሩጫ በኢትዮጵያ ውድድር በሠላም መጠናቀቁን ፖሊስ አስታወቀ,\n241\n\n\n19ኛው ታላቁ ሩጫ በኢትዮጵያ ውድድር በሠላም መጠናቀቁን...,https://www.ethiopianregistrar.com/amharic/pag...,297


In [3]:
def clean_text(text):
    '''Clean text by replacing non-string values with an empty string'''
    if (not text or text == -1):
        return '<UNK>'
    return text

In [4]:
df[['title', 'content']] = df[['title', 'content']].applymap(clean_text)
df = df[df['title'].notna() & (df['title'] != -1)]
df = df[df['content'].notna() & (df['title'] != -1)]

# Remove rows where 'short' is 'nan' (as a string)
df = df[df['content'] != 'nan']

df

,title,content,link,id
0,ነፃ ያልወጣች ሠንደቅ (መስፍን ማሞ ተሰማ),\n233\n\n\nአዎ፤ አውሮፓዊው ቅኝ ገዢ ጣሊያን አድዋ ላይ ድል ሆኗል...,https://www.ethiopianregistrar.com/amharic/pag...,181
1,title,content,link,id
2,ግዕዝ ኦሮምኛ ጨምሮ ለሁሉም የኢትዮጵያ ቋንቋዎች ብቃት እንዳለው ዶ/ር አ...,\n331\n\n\n“አገርኛ ቋንቋን ከኮምፒዩተር ያዋሃዱ ሳይንቲስት!” በሚ...,https://www.ethiopianregistrar.com/amharic/pag...,313
3,ከ OPDO ማዕከላዊ ኮሚቴ አፈትልከው የወጡት መረጃዎች እንደሚጠቁሙት አቶ...,\n2.1k\n\n\nጥር 26 ቀን 2008 ዓ.ም\nእንደ OPDO የውስጥ ም...,https://www.ethiopianregistrar.com/amharic/pag...,201
4,ባሕር ዳር ላይ በተደረገው የአፍሪካ ዋንጫ ማጣሪያ ጨዋታ ኢትዮጵያ ኮትዲቯ...,\n361\n\n\n\n\nጨዋታውን አስመልክቶ አሰልጣኙ፣ ተጨዋቾች እና ደጋ...,https://www.ethiopianregistrar.com/amharic/pag...,33
...,...,...,...,...
24093,“ከዚህ በፊት የኢትዮጵያ እስር ቤቶች ይታወቁ የነበረው በአካል ላይ ጥቃት...,\n753\n\n\n• ኢትዮጵያ ውስጥ ከአንድ ዓመት በፊት አዲስ የፖለቲከ ...,https://www.ethiopianregistrar.com/amharic/pag...,30
24094,ለባልደራስ፣ለኢዜማ፣ ለኦሮሞ ፓርቲ፣ ወይንስ ለፒፒ(ደኢህዴን)? የጉራጌ ህ...,\n361\n\n\nበ ሸምሱ በረዳ\n\nእንደሚታወቀው ቀጣዩ ሃገር አቀፍ ም...,https://www.ethiopianregistrar.com/amharic/pag...,65
24095,ሁለቱ ዋርዲያዎች፤ ጆቤና ጻድቃን ገብረተንሳይ በባህር በር አጀንዳ – መ...,\n501\n\n\nከተወሰ ግዜ ወዲህ አበበ ተክለ ሃይማኖት እና ጻድቃን ስ...,https://www.ethiopianregistrar.com/amharic/pag...,76
24096,ታላቁ ሩጫ በኢትዮጵያ ውድድር በሠላም መጠናቀቁን ፖሊስ አስታወቀ,\n241\n\n\n19ኛው ታላቁ ሩጫ በኢትዮጵያ ውድድር በሠላም መጠናቀቁን...,https://www.ethiopianregistrar.com/amharic/pag...,297


In [5]:
import re
class normalize(object):
    expansion_file_dir='' # assume you have file with list of short forms with their expansion as gazeter
    short_form_dict={}
    # Constructor
    def __init__(self):
       self.short_form_dict={}


        #method to normalize character level missmatch such as ጸሀይ and ፀሐይ
    def normalize_char_level_missmatch(self,input_token):
        rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
        rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
        rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
        rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
        rep5=re.sub('[ሕኅ]','ህ',rep4)
        rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
        rep7=re.sub('[ሠ]','ሰ',rep6)
        rep8=re.sub('[ሡ]','ሱ',rep7)
        rep9=re.sub('[ሢ]','ሲ',rep8)
        rep10=re.sub('[ሣ]','ሳ',rep9)
        rep11=re.sub('[ሤ]','ሴ',rep10)
        rep12=re.sub('[ሥ]','ስ',rep11)
        rep13=re.sub('[ሦ]','ሶ',rep12)
        rep14=re.sub('[ዓኣዐ]','አ',rep13)
        rep15=re.sub('[ዑ]','ኡ',rep14)
        rep16=re.sub('[ዒ]','ኢ',rep15)
        rep17=re.sub('[ዔ]','ኤ',rep16)
        rep18=re.sub('[ዕ]','እ',rep17)
        rep19=re.sub('[ዖ]','ኦ',rep18)
        rep20=re.sub('[ጸ]','ፀ',rep19)
        rep21=re.sub('[ጹ]','ፁ',rep20)
        rep22=re.sub('[ጺ]','ፂ',rep21)
        rep23=re.sub('[ጻ]','ፃ',rep22)
        rep24=re.sub('[ጼ]','ፄ',rep23)
        rep25=re.sub('[ጽ]','ፅ',rep24)
        rep26=re.sub('[ጾ]','ፆ',rep25)
        #Normalizing words with Labialized Amharic characters such as በልቱዋል or  በልቱአል to  በልቷል
        rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
        rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
        rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
        rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
        rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
        rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
        rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
        rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
        rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
        rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
        rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
        rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
        rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
        rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
        rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
        rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
        rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
        rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
        rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
        rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
        rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
        rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ

        return rep48

     #replacing any existance of special character or punctuation to null
    def remove_punc_and_special_chars(self,sentence_input): # puct in amh =፡።፤;፦፧፨፠፣
        normalized_text = re.sub('[\!\@\#\$\%\^\«\»\&\*\(\)\…\[\]\{\}\;\“\”\›\’\‘\"\'\:\,\.\‹\/\<\>\?\\\\|\`\´\~\-\=\+\፡\።\፤\;\፦\፥\፧\፨\፠\፣]', '',sentence_input)
        return normalized_text

    #remove all ascii characters and Arabic and Amharic numbers
    def remove_ascii_and_numbers(self,text_input):
        rm_num_and_ascii=re.sub('[A-Za-z0-9]','',text_input)
        rm_num_and_ascii=re.sub('-','',rm_num_and_ascii)
        return re.sub('[\'\u1369-\u137C\']+','',rm_num_and_ascii)



Text Preprocessing

In [6]:
import re
preprocess_class = normalize()
def preprocess_text(text):
    """Preprocesses text by applying various cleaning and normalization techniques."""
    if(text and text != -1 ):

      text=preprocess_class.normalize_char_level_missmatch(text)
      text=preprocess_class.remove_punc_and_special_chars(text)
      text=preprocess_class.remove_ascii_and_numbers(text)
      text = text.replace("\n", "")






      return text

df["title"] = df["title"].apply(preprocess_text)
df["content"] = df["content"].apply(preprocess_text)
df
df[['title', 'content']] = df[['title', 'content']].applymap(clean_text)
df = df[df['title'].notna() & (df['title'] != -1)]
df = df[df['content'].notna() & (df['title'] != -1)]

# Remove rows where 'short' is 'nan' (as a string)
df = df[df['content'] != 'nan']

df

,title,content,link,id
0,ነፃ ያልወጣች ሰንደቅ መስፍን ማሞ ተሰማ,አዎ አውሮፓዊው ቅኝ ገዢ ጣሊያን አድዋ ላይ ድል ሆኗል በ አም በኢትዮጵያ...,https://www.ethiopianregistrar.com/amharic/pag...,181
1,<UNK>,<UNK>,link,id
2,ግእዝ ኦሮምኛ ጨምሮ ለሁሉም የኢትዮጵያ ቋንቋዎች ብቃት እንዳለው ዶር አበ...,አገርኛ ቋንቋን ከኮምፒዩተር ያዋሀዱ ሳይንቲስት በሚል ርእስ አዲስ ዘመን ...,https://www.ethiopianregistrar.com/amharic/pag...,313
3,ከ ማእከላዊ ኮሚቴ አፈትልከው የወጡት መረጃዎች እንደሚጠቁሙት አቶ አባ ...,ጥር ቀን አምእንደ የውስጥ ምንጮች ገለፃ ከታች ስማቸው የተጠቀሰው የ...,https://www.ethiopianregistrar.com/amharic/pag...,201
4,ባህር ዳር ላይ በተደረገው የአፍሪካ ዋንጫ ማጣሪያ ጨዋታ ኢትዮጵያ ኮትዲቯ...,ጨዋታውን አስመልክቶ አሰልጣኙ ተጨዋቾች እና ደጋፊዎች አስተያዬታቸውን ሰጥ...,https://www.ethiopianregistrar.com/amharic/pag...,33
...,...,...,...,...
24093,ከዚህ በፊት የኢትዮጵያ እስር ቤቶች ይታወቁ የነበረው በአካል ላይ ጥቃት ...,• ኢትዮጵያ ውስጥ ከአንድ አመት በፊት አዲስ የፖለቲከ ምእራፍ ተከፍቷል ...,https://www.ethiopianregistrar.com/amharic/pag...,30
24094,ለባልደራስለኢዜማ ለኦሮሞ ፓርቲ ወይንስ ለፒፒደኢህዴን የጉራጌ ህዝብ በቀጣ...,በ ሸምሱ በረዳእንደሚታወቀው ቀጣዩ ሀገር አቀፍ ምርጫ ሊካሄድ እንደሆነ ተ...,https://www.ethiopianregistrar.com/amharic/pag...,65
24095,ሁለቱ ዋርዲያዎች ጆቤና ፃድቃን ገብረተንሳይ በባህር በር አጀንዳ – መስ...,ከተወሰ ግዜ ወዲህ አበበ ተክለ ሀይማኖት እና ፃድቃን ስለ ባህር በር እና...,https://www.ethiopianregistrar.com/amharic/pag...,76
24096,ታላቁ ሩጫ በኢትዮጵያ ውድድር በሰላም መጠናቀቁን ፖሊስ አስታወቀ,ኛው ታላቁ ሩጫ በኢትዮጵያ ውድድር በሰላም መጠናቀቁን የአዲስ አበባ ፖሊስ...,https://www.ethiopianregistrar.com/amharic/pag...,297


In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['content'],df['title'],test_size=0.3,random_state=0,shuffle=True)

In [8]:
print(len(x_train))
print(len(x_test))

16868
7230


In [9]:
SOS_token = 0
EOS_token = 1

input_index2word = {}
input_word2index={0: "SOS", 1: "EOS"}
output_index2word ={}
output_word2index = {0: "SOS", 1: "EOS"}
input_n_words = [2]
output_n_words = [2]
def addSentence(word2index,index2word,n_words, sentence):
  for word in sentence.split(' '):
    if word not in word2index:
      word2index[word] = n_words[0]
      index2word[n_words[0]] = word
      n_words[0] += 1







In [10]:
def readLangs(text, summary, reverse=False):

    text=np.array(text)
    summary=np.array(summary)
    pairs = [[text[i],summary[i]] for i in range(len(text))]


    return  pairs

In [11]:
def prepareData(lang1, lang2, reverse=False):
    pairs = readLangs(lang1, lang2, reverse)
    for pair in pairs:
        addSentence(input_word2index,input_index2word,input_n_words,pair[0])
        addSentence(output_word2index,output_index2word,output_n_words,pair[1])
    return pairs

In [12]:
import random
pairs = prepareData(x_train, y_train , False)
print(pairs[1])
print(random.choice(pairs))

['ጥር  ቀን  አምእንደ  የውስጥ ምንጮች ገለፃ ከታች ስማቸው የተጠቀሰው የወያኔው  ድርጅት ከፍተኛ የስራ አመራሮች ከስራ ገበታቸውና ከሀላፊነታቸው ታግዷልኛ አቶ አባ ዱላ ገመዳ የ  ማእከላዊ ኮሚቴ አባልና የተወካዮች ምክር ቤት አፈ ጉባኤኛ አቶ ሙክታር ከድር የ  ማእከላዊ ኮሚቴ አባልና የኦሮሚያ ክልላዊ መንግስት ፕሬዚዳንትኛ አቶ ለማ መገርሳ የ  ማእከላዊ ኮሚቴ አባልና የኦሮሚያ ክልላዊ መንግስት አፈ ጉባኤኛ አቶ ዳባ ደበሌ የ  ማእከላዊ ኮሚቴ አባልና የፅህፈት ቤቱ ተጠሪኛ አቶ አብይ አህመድ የሳይንስና ቴክኖሎጂ ሚኒስትር ናቸው\xa0 ', 'ከ  ማእከላዊ ኮሚቴ አፈትልከው የወጡት መረጃዎች እንደሚጠቁሙት አቶ አባ ዱላ ገመዳ ጨምሮ ሌሎች አራት የወያኔ  ከሀላፊነታቸው ሊታገዱ ነው']
['ካሀገር አልወጣም ግን  ምክትሎቹ ተሰውረዋልየአባይ ግድብ ስአስኪያጂ ኢንጂነር ስመኘው በቀለከአባይ ግድብ ስአስኪያጂ ኢንጂነር ስመኘው በቀለ በምክትልነት ሲያግዙ የነበሩ  ኢንጂነሮች የግድቡ ከፍተኛ ሀላፊወች የ ቀን እረፍት ወስደው እስከዛሬ አልተመለሱም ቱም ኢንጂነሮች ዛሬ ኛ ቀናቸው ቢሆንም ከምጥ ይግቡ ስምጥ እስካሁን ያሉበት ሁኔታ አይታወቅምከሀገር ይወጡ በሀገር ውስጥ ይኑሩ የታወቀ ነገር ባይኖርም የእጂ ስልካቸው አይሰራም ኢንጂነር ስመኘው በቀለ በተሰወሩት ቱ ኢንጂነሮች ምክንያት ከትናንት በስቲያ እና ትናንት ጥቅምት  አም ከህወሀት የመጡትን የደህንነት ተወካዩች ተቀብሎ ያነጋገራቸው ሲሆን ዛሬም በቢሮው ዙሪያ ከፍተኛ ደህንነቶች ተወጥሮ ውሏልየግድቡ ስራ ሙሉ በሙሉ በሚባል ሁኔታ የቆመ ሲሆን በጉባ የግድቡ ሳይት ታጣቂ ወታደሮች እንጂ ሰራተኛ አይታይምባለፉት  አመት ለግድቡ ስራ የገቡ ማሽነሪዎች  ትልልቅ መሳሪያዎች ተጭነው ከወጡት ውጭ በሳይቱ ተቆልፎባቸው በምእራብ እዝ አጋዚ ወ

In [13]:
def tensorForSingleSentence(word2index, sentence):

    indexes = [word2index[word] for word in sentence.split(' ')]
    indexes.append(EOS_token)

    index = list(indexes)
    return torch.tensor(index, dtype=torch.long, device=device).view(-1, 1)

def tensorsForPair(input_word2index,output_word2index,pair):

    input=tensorForSingleSentence(input_word2index,pair[0])
    output=tensorForSingleSentence(output_word2index,pair[1])
    return (input,output)




In [14]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.LSTM = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.LSTM(output, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),torch.zeros(1, 1, self.hidden_size, device=device))

In [16]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.LSTM = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.LSTM(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [17]:
MAX_LENGTH=200

In [18]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size*2 , self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size*2 , self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.LSTM = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)


    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)


        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)

        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.LSTM(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [19]:
teacher_forcing_ratio = 0.5
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0


    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

        temp = encoder_output[0, 0]

        encoder_outputs[ei] = temp

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    for di in range(target_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(
      decoder_input, decoder_hidden, encoder_outputs)


      topv, topi = decoder_output.topk(1)

      decoder_input = topi.squeeze().detach()

      loss += criterion(decoder_output, target_tensor[di])

      if decoder_input.item() == EOS_token:
             break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [22]:
import numpy as np
import time



def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    print("Training....")
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsForPair(input_word2index,output_word2index,random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        if iter% 10 == 0:
            print(iter,"/",n_iters + 1)
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        input_length = input_tensor.size(0)
        if(input_length > 200):

          continue

        loss = train(input_tensor, target_tensor, encoder,decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss









hidden_size = 300
encoder1 = EncoderRNN(input_n_words[0], hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_n_words[0], dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 100, print_every=10)

Training....
10 / 101
20 / 101
30 / 101
40 / 101
50 / 101
60 / 101
70 / 101
80 / 101
90 / 101
100 / 101


In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
torch.save(encoder1.state_dict(), './encoder.w')
torch.save(attn_decoder1.state_dict(), './attension.p')


The output is poor, since we have only done the embedding from our dataset and lack our train iteration due to colab crashing while training the model

In [24]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorForSingleSentence(input_word2index, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):

            encoder_output, encoder_hidden = encoder(input_tensor[ei],encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)


        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]
pair = random.choice(pairs)
print(pair)
output_words , attentions = evaluate(encoder1 , attn_decoder1, pair[0])
output_sentence = ' '.join(output_words)
output_sentence

['ሳሞራ የኑስ ተነሱ  የሚተካቸውን ሹክ እንበላችሁ ', 'ሳሞራ የኑስ ተነሱ  የሚተካቸውን ሹክ እንበላችሁ']


'በተማሪዎች <EOS>'